<a href="https://colab.research.google.com/github/JVMergulho/DeepLearningExamples/blob/main/WineQualiyRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using the Wine Quality dataset form UCI ML Repository for a regression problem

In [75]:
#imports
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo # Biblioteca para carregar datasets UCI

from google.colab import files

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler

In [76]:
# fetch dataset
wine_quality = fetch_ucirepo(id=186)

# data (as pandas dataframes)
X = wine_quality.data.features.to_numpy()
y = wine_quality.data.targets.to_numpy()

# metadata
print(wine_quality.metadata)

# variable information
print(wine_quality.variables)

{'uci_id': 186, 'name': 'Wine Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/186/wine+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/186/data.csv', 'abstract': 'Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 4898, 'num_features': 11, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['quality'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Wed Nov 15 2023', 'dataset_doi': '10.24432/C56S3T', 'creators': ['Paulo Cortez', 'A. Cerdeira', 'F. Almeida', 'T. Matos', 'J. Reis'], 'intro_paper': {'ID': 252, 'type': 'NATIVE', 'title': 'Modeling wine preferences

In [77]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

In [78]:
dataset = TensorDataset(X_tensor, y_tensor)

#Definir tamanhos das divisões
train_size = int(0.7 * len(dataset))  # 70% para treino
val_size = int(0.15 * len(dataset))   # 15% para validação
test_size = len(dataset) - train_size - val_size  # 15% para teste

#Dividir dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [79]:
print(f"Tamanho do treino: {len(train_dataset)}")
print(f"Tamanho da validação: {len(val_dataset)}")
print(f"Tamanho do teste: {len(test_dataset)}")

Tamanho do treino: 4547
Tamanho da validação: 974
Tamanho do teste: 976


In [80]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [81]:
class WineModel(nn.Module):
  def __init__(self, input_size):
    super(WineModel, self).__init__()
    self.Model = nn.Sequential(
        nn.Linear(input_size, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32,1)
    )

  def forward(self, x):
      return self.Model(x)


In [82]:
#Instanciar modelo
input_size = X.shape[1]  # Número de features
model = WineModel(input_size)

In [83]:
# Definir função de perda e otimizador
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [84]:
patience = 10  # Número de épocas sem melhora antes de parar
num_max_epochs = 60 # Número máximo de épocas
best_val_loss = float('inf')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

WineModel(
  (Model): Sequential(
    (0): Linear(in_features=11, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [85]:
for epoch in range(num_max_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Avaliação no conjunto de validação
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            y_val_pred = model(X_val)
            val_loss += criterion(y_val_pred, y_val).item()

    # Exibir progresso
    print(f"Época {epoch+1}/{num_epochs} - Perda treino: {train_loss/len(train_loader):.4f}, "
          f"Perda validação: {val_loss/len(val_loader):.4f}")

    # Early Stopping: verificar se a perda de validação melhorou
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0  # Resetar contador
        best_model = model.state_dict()  # Salvar o melhor modelo
    else:
        counter += 1
        if counter >= patience:
            print(f"\nParando treinamento após {epoch+1} épocas. Early Stopping ativado.")
            break

# Carregar o melhor modelo encontrado
model.load_state_dict(best_model)
print("Modelo carregado com os melhores pesos salvos!")

Época 1/50 - Perda treino: 12.1322, Perda validação: 1.9179
Época 2/50 - Perda treino: 1.6144, Perda validação: 1.3145
Época 3/50 - Perda treino: 1.2021, Perda validação: 1.0501
Época 4/50 - Perda treino: 0.9453, Perda validação: 0.8520
Época 5/50 - Perda treino: 0.7808, Perda validação: 0.7539
Época 6/50 - Perda treino: 0.6553, Perda validação: 0.6554
Época 7/50 - Perda treino: 0.5735, Perda validação: 0.5929
Época 8/50 - Perda treino: 0.5288, Perda validação: 0.5751
Época 9/50 - Perda treino: 0.5073, Perda validação: 0.5491
Época 10/50 - Perda treino: 0.4932, Perda validação: 0.5299
Época 11/50 - Perda treino: 0.4880, Perda validação: 0.5309
Época 12/50 - Perda treino: 0.4792, Perda validação: 0.5245
Época 13/50 - Perda treino: 0.4846, Perda validação: 0.5402
Época 14/50 - Perda treino: 0.4769, Perda validação: 0.5094
Época 15/50 - Perda treino: 0.4745, Perda validação: 0.5172
Época 16/50 - Perda treino: 0.4616, Perda validação: 0.5085
Época 17/50 - Perda treino: 0.4638, Perda valida

In [86]:
# Colocar o modelo em modo de avaliação
model.eval()

# Obter os 10 primeiros exemplos do conjunto de teste
X_test_sample, y_test_sample = next(iter(test_loader))  # Pega o primeiro batch
X_test_sample, y_test_sample = X_test_sample[:10].to(device), y_test_sample[:10].to(device)  # Seleciona os 10 primeiros

# Fazer previsões
with torch.no_grad():
    y_test_pred_sample = model(X_test_sample)

# Calcular a perda nos 10 primeiros exemplos
loss_sample = criterion(y_test_pred_sample, y_test_sample).item()

print(f"Perda nos 10 primeiros exemplos do conjunto de teste: {loss_sample:.4f}")

# Mostrar previsões e valores reais
print("\nValores Reais:", y_test_sample.cpu().numpy().flatten())
print("Previsões:", y_test_pred_sample.cpu().numpy().flatten())


Perda nos 10 primeiros exemplos do conjunto de teste: 0.1444

Valores Reais: [5. 7. 6. 5. 4. 6. 6. 6. 6. 6.]
Previsões: [5.050444  6.6952543 6.330924  5.2746954 4.8724217 6.259461  5.774275
 5.9681935 5.473256  5.9273844]


In [87]:
# Caminho para salvar o modelo
model_path = "wine_quality_model.pth"

# Salvar os pesos do modelo
torch.save(model.state_dict(), model_path)

print(f"Modelo salvo em: {model_path}")

Modelo salvo em: wine_quality_model.pth


In [88]:
# Fazer o download do modelo salvo
# files.download(model_path)